In [ ]:
! pip install --upgrade codetiming numpy pandas census us mechanicalsoup geopandas pandas_bokeh torch
get_ipython().kernel.do_shutdown(True)

In [ ]:
project_id = 'redistricting-361203'
repo = 'voting_predictor'
mount_path = '/content/drive'
root_path = f'{mount_path}/MyDrive/gerrymandering/2022-10'
repo_path = f'{root_path}/{repo}'
data_path = f'{root_path}/data'

from google.colab import auth, drive
from google.cloud import bigquery
auth.authenticate_user()
drive.mount(mount_path)
client = bigquery.Client(project=project_id)
%cd {repo_path}
%load_ext google.colab.data_table
%load_ext autoreload
%autoreload

In [ ]:
from acs5 import *

In [ ]:
%ls